In [1]:
import pandas as pd
import ast

In [2]:
df_stops = pd.read_csv("1960-stops-final.csv")  

In [3]:
df_stops.drop(['Index', "index", "station_id", "stop_id"], axis=1, inplace=True)

In [4]:
df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   stop_description     1400 non-null   object
 1   type                 1400 non-null   object
 2   in_lines             1400 non-null   object
 3   combined_identifier  361 non-null    object
 4   combined_location    1400 non-null   object
 5   original_name        1400 non-null   object
dtypes: object(6)
memory usage: 65.8+ KB


In [5]:
df_line = pd.read_csv("line_data_1960.csv")  

In [6]:
df_line['line_name'] = df_line['line_name'].astype(str).str.strip()
df_line['last_change'] = df_line['last_change'].astype(str).str.strip()
df_line['Stops'] = df_line['Stops'].astype(str).str.strip()
df_line['start_stop'] = df_line['start_stop'].astype(str).str.strip()
df_line['year'] = df_line['year'].astype(int)
df_line['Frequency'] = pd.to_numeric(df_line['Frequency'], errors='coerce').fillna(0).astype(int)
df_line['Length (time)'] = pd.to_numeric(df_line['Length (time)'], errors='coerce').fillna(0).astype(int)
df_line['Length (km)'] = pd.to_numeric(df_line['Length (km)'], errors='coerce').fillna(0).astype(int)

In [7]:
# Define a function to remove leftover parentheses from a column
def remove_double_whitespace(text):
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("–", "-")
    text = text.replace(" - ", " - ")
    return text

# Apply the function to the 'line_stops' column
df_line['Stops'] = df_line['Stops'].apply(remove_double_whitespace)
df_line['Stops'] = df_line['Stops'].str.replace(r'\s*–\s*', ' - ', regex=True)

In [8]:
mask = df_line['Stops'].str.contains(" - ").fillna(True)
none_df = df_line[~mask]
none_df

,Stops,Frequency,Length (time),Length (km),line_name,type,east-west,year,start_stop,last_change


In [9]:
# Filter rows where 'line_stops' is not a string
non_string_rows = df_line[~df_line['Stops'].apply(lambda x: isinstance(x, str))]
non_string_rows

,Stops,Frequency,Length (time),Length (km),line_name,type,east-west,year,start_stop,last_change


In [10]:
### Creating related tables

In [11]:
def extract_first_start_stop(string):
    stations = string.split(" - ")
    first_station = stations[0]
    last_station = stations[-1]
    return f"{first_station}<> {last_station}"

In [12]:
def create_line_df(df):
    line_df = pd.DataFrame({
        'line_id': range(1, 1+len(df)),
        'year': df['year'],
        'line_name': df['line_name'],
        'type': df["type"],
        "start_stop": df['Stops'].apply(extract_first_start_stop),
        "Length (time)": df['Length (time)'],
        "Length (km)": df['Length (km)'],
        "last_change": df['last_change'],
        "east_west": df['east-west'],
        "Frequency": df['Frequency']
    })
    return line_df

In [13]:
line_df = create_line_df(df_line)

# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_df['line_id'] = '1960' + line_df['line_id'].astype(str)
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),Length (km),last_change,east_west,Frequency
0,19601,1960,1,strassenbahn,SchillingBrücke (Ostbahnhof)<> Am Kupfergraben,33,7,nan,Ost,20
1,19602,1960,11,strassenbahn,Köpenicker Strasse Ecke Adalbertstrasse<> Oran...,23,5,nan,Ost,20


In [14]:
# Reset the index and rename the index column to 'stop_id'
df_stops.reset_index(inplace=True)
df_stops.rename(columns={'index': 'stop_id'}, inplace=True)
df_stops.rename(columns={'original_name': 'stop_name'}, inplace=True)


In [15]:
for index, row in df_stops.iterrows():
    df_stops.at[index, "stop_id"] = row["stop_id"] + 1

In [16]:
df_stops

,stop_id,stop_description,type,in_lines,combined_identifier,combined_location,stop_name
0,1,4. Ring,bus,{'A53'},NaN,"52.42250630689789, 13.2869490566549",4. Ring
1,2,Adamstrasse Ecke Pichelsdorfer Strasse,bus,"{'A34', 'A35'}",NaN,"52.521029117218866, 13.19780699419913",Adamstrasse
2,3,Berlin-Adlershof station,s-bahn,"{'KBS 103', 'KBS 100a'}",Q323551,"52.434722222222,13.541388888889",Adlershof
3,4,Adolf-Scheidt-Platz,strassenbahn,{'95'},NaN,"52.478194856980814, 13.37836902238447",Adolf-Scheidt-Platz
4,5,Afrikanische Straße,u-bahn,"{'C I', 'C II'}",Q559239,"52.560027777778,13.334633333333",Afrikanische Str.
...,...,...,...,...,...,...,...
1395,1396,Waldschänke,bus,{'A22'},NaN,"52.443318139967566, 13.68593963431219",Waldschänke
1396,1397,Wilhelmstrasse Ecke Pichelsdorferstrasse,bus,"{'A35', 'A34'}",NaN,"52.52799171469617, 13.19527000737161",Wilhelmstrasse Ecke Pichelsdorferstrasse
1397,1398,Köpenicker Strasse Ecke Manteuffelstrasse,bus,{'A28'},NaN,"52.505752241611106, 13.431927610647389",Köpenicker Strasse Ecke Manteuffelstrasse
1398,1399,Lichterfelde Süd Lindenstrasse,strassenbahn,{'78'},NaN,"52.41370384479345, 13.30156889737024",Lichterfelde Süd Lindenstrasse


In [17]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
df_stops['stop_id'] = '1960' + df_stops['stop_id'].astype(str)
df_stops.head(2)

,stop_id,stop_description,type,in_lines,combined_identifier,combined_location,stop_name
0,19601,4. Ring,bus,{'A53'},NaN,"52.42250630689789, 13.2869490566549",4. Ring
1,19602,Adamstrasse Ecke Pichelsdorfer Strasse,bus,"{'A34', 'A35'}",NaN,"52.521029117218866, 13.19780699419913",Adamstrasse


In [18]:
def create_line_stops_df(df):
    line_stops = df['Stops'].str.split(' - ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='stop_name')

    line_stops['stop_order'] = line_stops.groupby('index').cumcount()
    #index starts from 0 so it looks like 1 row is missing but this is not true

    # Clean the 'Stop Name' column by removing whitespace and non-breaking spaces
    line_stops['stop_name'] = line_stops['stop_name'].str.replace(u'\xa0', ' ').str.strip()

    # reset index so that it can be used for foreign key

    return line_stops

In [19]:
line_stops_df = create_line_stops_df(df_line)

In [20]:
for index, row in line_stops_df.iterrows():
    line_stops_df.at[index, "index"] = row["index"] + 1
line_stops_df.rename(columns={'index': 'line_id'}, inplace=True)

# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_stops_df['line_id'] = '1960' + line_stops_df['line_id'].astype(str)
line_stops_df.head(2)

,line_id,stop_name,stop_order
0,19601,SchillingBrücke (Ostbahnhof),0
1,19601,U-Bhf. Strausberger Platz,1


In [21]:
# get a list of unique Line IDs in the line_stops DataFrame
line_ids = line_stops_df['line_id'].unique()

# loop through each unique Line ID
for line_id in line_ids:
    # get the first and last Stop Name for this Line ID
    first_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[0]
    last_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[-1]
    
    # get the corresponding Start-Stop value from line_df for this Line ID
    start_stop = line_df.loc[line_df['line_id'] == line_id, 'start_stop'].iloc[0]
    
    # check if the first and last Stop Names are contained in the Start-Stop value
    if first_stop_name not in start_stop or last_stop_name not in start_stop:
        print(f"Quality control check failed for line_id {line_id}.")

# I checked that this works using the following code:
# line_stops_df.loc[0, "stop_name"] = "test"
# and got the right result: Quality control check failed for line_id 1001.


In [22]:
line_df

,line_id,year,line_name,type,start_stop,Length (time),Length (km),last_change,east_west,Frequency
0,19601,1960,1,strassenbahn,SchillingBrücke (Ostbahnhof)<> Am Kupfergraben,33,7,nan,Ost,20
1,19602,1960,11,strassenbahn,Köpenicker Strasse Ecke Adalbertstrasse<> Oran...,23,5,nan,Ost,20
2,19603,1960,13,strassenbahn,"Klingenberg, Blockdammweg<> Oderberger Strasse",37,9,nan,Ost,7
3,19604,1960,15,strassenbahn,Daimlerstrasse<> Schulenburgpark,36,0,nan,West,15
4,19605,1960,2,strassenbahn,Wolliner Strasse<> Spreewaldplatz,70,0,nan,West,15
...,...,...,...,...,...,...,...,...,...,...
152,1960153,1960,KBS 104,s-bahn,Oranienburg<> Wannsee,0,0,08.02.1950,NaN,0
153,1960154,1960,KBS 104a,s-bahn,Zehlendorf<> Düppel-Kleinmachnow,0,0,15.06.1948,NaN,0
154,1960155,1960,KBS 104b,s-bahn,Wannsee<> Stahnsdorf,0,0,11.09.1954,NaN,0
155,1960156,1960,KBS 105,s-bahn,Velten (Mark)<> Rangsdorf,0,0,18.05.1952,NaN,0


In [23]:
def add_type(line_stops, line_df):
    # Assuming line_id is the common column between line_stops and line_df
    merged_df = pd.merge(line_stops, line_df[['line_id', 'type', "line_name"]], on='line_id', how='left')
    
    # Rename the 'type' column from line_df to 'type_from_line_df' to avoid conflicts
    merged_df.rename(columns={'type': 'type'}, inplace=True)
    
    # Drop the 'type_from_line_df' column if it's not needed in the final result
    # merged_df.drop(columns=['type_from_line_df'], inplace=True)
    
    return merged_df

In [24]:
line_stops_df = add_type(line_stops_df, line_df)

In [25]:
line_stops_df

,line_id,stop_name,stop_order,type,line_name
0,19601,SchillingBrücke (Ostbahnhof),0,strassenbahn,1
1,19601,U-Bhf. Strausberger Platz,1,strassenbahn,1
2,19601,Stalinallee,2,strassenbahn,1
3,19601,Leninplatz,3,strassenbahn,1
4,19601,"Alexanderplats, Memhardstr.",4,strassenbahn,1
...,...,...,...,...,...
2103,1960157,Südende,20,s-bahn,KBS 106
2104,1960157,Lankwitz,21,s-bahn,KBS 106
2105,1960157,Lichterfelde Ost,22,s-bahn,KBS 106
2106,1960157,Lichterfelde Süd,23,s-bahn,KBS 106


In [26]:
# Function to convert string representation of set to list
def convert_to_list(value):
    try:
        # Use ast.literal_eval to safely convert string to set
        items = ast.literal_eval(value)
        # Convert the set to a list
        return list(items)
    except (SyntaxError, ValueError):
        # Handle the case where literal_eval fails (e.g., invalid string)
        return [value]

# Apply the function to the 'in_lines' column
df_stops['in_lines'] = df_stops['in_lines'].apply(convert_to_list)

In [27]:
df_stops['in_lines'] = df_stops['in_lines'].apply(lambda x: ','.join(map(str, x)))

In [28]:
df_stops.head()

,stop_id,stop_description,type,in_lines,combined_identifier,combined_location,stop_name
0,19601,4. Ring,bus,A53,NaN,"52.42250630689789, 13.2869490566549",4. Ring
1,19602,Adamstrasse Ecke Pichelsdorfer Strasse,bus,"A34,A35",NaN,"52.521029117218866, 13.19780699419913",Adamstrasse
2,19603,Berlin-Adlershof station,s-bahn,"KBS 100a,KBS 103",Q323551,"52.434722222222,13.541388888889",Adlershof
3,19604,Adolf-Scheidt-Platz,strassenbahn,95,NaN,"52.478194856980814, 13.37836902238447",Adolf-Scheidt-Platz
4,19605,Afrikanische Straße,u-bahn,"C II,C I",Q559239,"52.560027777778,13.334633333333",Afrikanische Str.


In [29]:
def add_fk(line_stops_df, df_stops):
    # Create a new dataframe with the Stop Name and Stop ID columns
    stop_id_df = df_stops[['stop_name', 'stop_id', 'type', 'in_lines']]

    # Merge the line_stops_df and stop_id_df dataframes based on matching stop names and line_name condition
    line_stops_df = line_stops_df.merge(stop_id_df,
                                        left_on=['stop_name', 'type'],
                                        right_on=['stop_name', 'type'],
                                        how='left')

    # Filter the rows based on the condition that line_name is contained in in_lines
    line_stops_df = line_stops_df[line_stops_df.apply(lambda row: str(row['line_name']) in str(row['in_lines']), axis=1)]

    return line_stops_df

# Assuming line_stops_df and df_stops are your dataframes
# Replace 'stop_name', 'stop_id', 'type', and 'in_lines' with the actual column names you have

line_stops_df = add_fk(line_stops_df, df_stops)


In [30]:
# Calculate the difference between consecutive 'stop_order' values
line_stops_df['diff'] = line_stops_df['stop_order'].diff()

# Identify faulty rows where the difference is not 1 digit behind
faulty_rows = line_stops_df[(line_stops_df['diff'] != 1) & (line_stops_df['stop_order'] != 0)]
faulty_rows

,line_id,stop_name,stop_order,type,line_name,stop_id,in_lines,diff
541,196039,Lichterfelde Süd Lindenstrasse,14,strassenbahn,78,19601399,78,2.0
1140,196079,Alt-Kladow,13,bus,A34,196020,"A34,A35",2.0
1159,196080,Alt-Kladow,13,bus,A35,196020,"A34,A35",2.0
1171,196082,"Hohensch nhausen, Falkenberger Strasse",4,bus,A39,1960492,A39,2.0
1560,1960112,Nordend,5,bus,A7,1960779,A7,2.0


In [31]:
# convert 'Stop ID' column to numeric values, coercing errors to NaN
line_stops_df['stop_id'] = pd.to_numeric(line_stops_df['stop_id'], errors='coerce')

# check if all values in 'Stop ID' column are numeric
if line_stops_df['stop_id'].notnull().all():
    print("All values in 'stop_id' column are numeric")
else:
    print("There are non-numeric values in 'stop_id' column")
    print(line_stops_df[line_stops_df['stop_id'].isnull()])


All values in 'stop_id' column are numeric


In [32]:
line_stops_df.drop(['line_name', "in_lines", "type", "diff"], axis=1, inplace=True)

In [33]:
line_stops_df.to_csv("line_stops_1960-final.csv")
line_df.to_csv("line_df_1960-final.csv")
df_stops.to_csv("stops_df_1960-final.csv")